# Colaboratory setup

In [1]:
# hdf5
# pytorch lightning
# wandb
# hydra
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd /drive/MyDrive

import os
import subprocess
import warnings
warnings.filterwarnings("ignore")

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)
if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex
!pip install ftfy==5.8
!pip install transformers
!pip install open_clip_torch

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, UnidentifiedImageError
import open_clip
import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import transformers
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    torch.backends.cudnn.benchmark = True
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

%cd "/content/drive/MyDrive/Colab Notebooks/mclip-finetune"

[Errno 2] No such file or directory: '/drive/MyDrive'
/Users/k1096/mclip-finetune


FileNotFoundError: [Errno 2] No such file or directory: 'nvcc'

# Finetuning

In [54]:
import importlib

from torch.utils.data import DataLoader

import hdf5
importlib.reload(hdf5)


HDF5_FILE = 'coco2014.h5'
CAPTIONS_DIR = 'captions'
CAPTIONS_TRAIN_EN = 'captions_train2014.json'
CAPTIONS_VAL_EN = 'captions_val2014.json'
CAPTIONS_TRAIN_JP = 'stair_captions_v1.2_train.json'
CAPTIONS_VAL_JP = 'stair_captions_v1.2_val.json'
IMAGES_TRAIN_DIR = 'train2014'
IMAGES_VAL_DIR = 'val2014'


dataset = hdf5.HDF5dataset(HDF5_FILE)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)
train_subsetloader = DataLoader(dataset, batch_size=100, shuffle=False)
val_subsetloader = DataLoader(dataset, batch_size=100, shuffle=True)

In [55]:
import finetune
importlib.reload(finetune)

device = 'cpu'

TEXT_MODEL_NAME = 'M-CLIP/XLM-Roberta-Large-Vit-B-16Plus'
IMAGE_MODEL_NAME, IMAGE_PRETRAINED_NAME = 'ViT-B-16-plus-240', "laion400m_e32"
text_model, tokenizer, image_model, preprocess, logit_scale = \
    finetune.get_models(TEXT_MODEL_NAME, IMAGE_MODEL_NAME, IMAGE_PRETRAINED_NAME)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [56]:
importlib.reload(finetune)

finetune.finetune(dataloader, train_subsetloader, val_subsetloader, text_model, tokenizer, image_model, preprocess, logit_scale, device, epochs=20)

initial rSum val:594.0, rSum train:581.0
torch.Size([16, 640])
torch.Size([16, 3, 224, 224])
torch.Size([16, 640])
torch.Size([16, 3, 224, 224])


KeyboardInterrupt: 